In [3]:
import kaggle

In [4]:
!kaggle competitions download -c ds1-tree-ensembles

100%|██████████████████████████████████████| 4.67M/4.67M [00:01<00:00, 3.09MB/s]

  0%|                                                | 0.00/295k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 295k/295k [00:00<00:00, 3.80MB/s]
 86%|████████████████████████████████▌     | 1.00M/1.17M [00:00<00:00, 2.53MB/s]
100%|██████████████████████████████████████| 1.17M/1.17M [00:00<00:00, 2.93MB/s]
  0%|                                               | 0.00/92.2k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 92.2k/92.2k [00:00<00:00, 5.62MB/s]
  0%|                                               | 0.00/23.0k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 23.0k/23.0k [00:00<00:00, 21.2MB/s]


In [5]:
!pwd

/home/mark/lambda/DS-Unit-4-Sprint-1-Tree-Ensembles/module1-decision-trees


"age";"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y



In [130]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from datetime import datetime

In [131]:
dd = pd.read_excel('LCDataDictionary.xlsx')

In [18]:
dd.LoanStatNew

0                                  acc_now_delinq
1                            acc_open_past_24mths
2                                      addr_state
3                                        all_util
4                                      annual_inc
5                                annual_inc_joint
6                                application_type
7                                     avg_cur_bal
8                                  bc_open_to_buy
9                                         bc_util
10                       chargeoff_within_12_mths
11                        collection_recovery_fee
12                     collections_12_mths_ex_med
13                                    delinq_2yrs
14                                    delinq_amnt
15                                           desc
16                                            dti
17                                      dti_joint
18                               earliest_cr_line
19                                     emp_length


In [42]:
df = pd.read_csv('train_features.csv')

In [86]:
df.head()

,id,member_id,loan_amnt,funded_amnt,term,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,int_rates
0,43373,NaN,12000,12000,36 months,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,16.02
1,24414,NaN,6300,6300,36 months,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,14.07
2,46723,NaN,4500,4500,36 months,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,7.21
3,24878,NaN,12000,12000,36 months,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,9.44
4,39038,NaN,12000,12000,36 months,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,22.35


In [ ]:
df.sec_app_earliest_cr_line.notnan()

In [139]:
rev_util_mean = df.revol_util.dropna().apply(lambda f: float(f[:-1])).mean()

In [140]:
df.revol_util = df.revol_util.apply(lambda f: rev_util_mean if isinstance(f, float) else float(f[:-1]))

In [133]:
df.earliest_cr_line = df.earliest_cr_line.apply(lambda d: datetime.strptime(d,'%b-%Y'))

In [88]:
max(df.term.str.len())
// ' 36 months'


10

In [97]:
df.term = df.term.apply(lambda t: int(t[1:3]))

In [98]:
df.term.head()

0    36
1    36
2    36
3    36
4    36
Name: term, dtype: int64

In [100]:
df.grade = df[['grade', 'sub_grade']].apply(lambda o: o[0] + '-' + o[1])

In [56]:
dfl = df.columns.tolist()

In [66]:
df.dtypes

id                                       int64
member_id                              float64
loan_amnt                                int64
funded_amnt                              int64
term                                    object
int_rate                                object
installment                            float64
grade                                   object
sub_grade                               object
emp_title                               object
emp_length                              object
home_ownership                          object
annual_inc                             float64
url                                    float64
desc                                   float64
purpose                                 object
title                                   object
zip_code                                object
addr_state                              object
dti                                    float64
delinq_2yrs                              int64
earliest_cr_l

In [105]:
max(df.emp_length.str.len())

9.0

In [111]:
print(f"|{df.emp_length.iloc[0]}|{df.emp_length.iloc[4]}|")

|4 years|10+ years|


In [127]:
def el(e):
    if isinstance(e, float) or e[0] == '<':
        return 0
    return int(e[0:(2 if e[1] == '0' else 1)])

In [128]:
df.emp_length = df.emp_length.apply(lambda e: el(e))

In [44]:
df_train_y =  pd.read_csv('train_labels.csv')

In [45]:
df_train_y.dtypes

id             int64
charged_off    int64
dtype: object

In [46]:
# df['y'] = df_train_y.charged_off

In [77]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
import numpy as np

In [39]:
# sns.set(style='ticks', color_codes=True)
# independant_columns = df.columns.drop('y')
# sns.pairplot(data=df,
#                   y_vars=['y'],
#                   x_vars=independant_columns)
# plt.show()

In [74]:
df['int_rates'] = df.int_rate.apply(lambda r: float(r[:-1]))

In [76]:
df.int_rates.head()

0    16.02
1    14.07
2     7.21
3     9.44
4    22.35
Name: int_rates, dtype: float64

In [103]:
df.drop(['sub_grade'], axis=1, inplace=True)
// 'int-rate'

In [141]:
 df.select_dtypes(exclude=[np.number]).columns

Index(['grade', 'emp_title', 'home_ownership', 'purpose', 'title', 'zip_code',
       'addr_state', 'earliest_cr_line', 'initial_list_status',
       'application_type', 'sec_app_earliest_cr_line', 'disbursement_method'],
      dtype='object')

In [85]:
df.zip_code.head()

0    280xx
1    920xx
2    906xx
3    441xx
4    751xx
Name: zip_code, dtype: object

In [75]:
X = imputer.fit_transform(df[['int_rates']])

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def classify_loans(max_depth):
    tree = DecisionTreeClassifier(max_depth=max_depth)
    tree.fit(df, d.train_y.chargedoff)
    plot_decision_regions(titanic_X, titanic_y, tree, legend=False)
    plt.axis((0,75,0,175))
    plt.show()